In [ ]:
!(nvidia-smi | grep -q "has failed") && echo "No GPU found!" || nvidia-smi 

In [ ]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

Import MNIST dataset.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/MNIST_data")

Visualize the input data. Notice that the `x_train` data is for visualization, we define training data in network later. 

In [ ]:
x_train = mnist.train.images[:55000,:]
x_train.shape

randomNum = random.randint(0,55000)
image = x_train[randomNum].reshape([28,28])
plt.imshow(image,cmap=plt.get_cmap('gray_r'))

### Helper functions

In [ ]:
def conv2d(x,W):
    return tf.nn.conv2d(input=x,filter=W,strides=[1,1,1,1],padding='SAME')

In [ ]:
def conv2d_withwb(x, k_w=3, k_h=3, input_channel=1, output_channel=8, stride=1, name='conv2d', reuse=False):
    """
    k_w: kernel width
    k_h: kernel height
    """
    with tf.variable_scope(name, reuse=reuse):
        w = tf.get_variable('w', [k_w, k_h, input_channel, output_channel],
                            initializer=tf.truncated_normal_initializer(stddev=0.02))
        b = tf.get_variable('b', [output_channel], initializer=tf.constant_initializer(0))

    return tf.nn.conv2d(x, w, strides=[1,stride,stride,1], padding='SAME') + b

In [ ]:
def avg_pool_2x2(x,stride=2):
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,stride,stride,1], padding='SAME')

In [ ]:
def deconv2d(x, k_w=3, k_h=3, output_shape=[1,1,1,1], stride=2, padding='SAME', name='deconv', reuse=False):
    with tf.variable_scope(name, reuse=reuse):
        w = tf.get_variable('w', [k_w, k_h, output_shape[-1], int(x.get_shape()[-1])], 
                            initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', [output_shape[-1]], initializer=tf.constant_initializer(0.1))
    
    return b + tf.nn.conv2d_transpose(x, w, output_shape=output_shape, strides=[1,stride,stride,1], padding=padding)

TO DO TASK: 

Build discriminator network

Input: One batch of images with each image having scale `[28,28]` So input size is `[batch_size,28,28]`

Output: A vector with dimension `[batch_size,1]`. (Discriminator outputs a scalar for each sample, so for one batch it outpus `[batch_size,1]`)

**Task 1.1**: 

Build network architecture use convolutional layer with `stride=1`, add `tf.nn.relu` as activation function. Use max pool to downsample.

---

**Task 2.1**: 

(Yes, this is **first** part of **task 2**. You will come back to this part after finishing **task 1**)

Build network architecture use convolutional layer with `stride=2`, add `tf.nn.relu` as activation function. **No** max pool.

---

**Task 3.1**:

(As above, this is, again, **first** part of **task 3**. Come back to it after finishing **task 2**)

Build network ONLY with 2 or 3 fully connect layer  (`tf.layers.dense`). Suggested number of hidden unit: 256.

---

Compare the results from the different methods.

In [ ]:
def discriminator(x,reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):            
        # First conv and pool layer
        W_conv1 = tf.get_variable('d_wconv1', [5, 5, 1, 8], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('d_bconv1', [8],          initializer=tf.constant_initializer(0))
        # W.x+b
        h_conv1 = tf.nn.leaky_relu(conv2d(x, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)

        #Second Conv and Pool Layers
        W_conv2 = tf.get_variable('d_wconv2', [5, 5, 8, 16], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2', [16],          initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)

        #First Fully Connected Layer
        W_fc1 = tf.get_variable('d_wfc1', [7 * 7 * 16, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [32],             initializer=tf.constant_initializer(0))
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*16])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

        #Second Fully Connected Layer
        W_fc2 = tf.get_variable('d_wfc2', [32, 1], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1],     initializer=tf.constant_initializer(0))

        #Final Layer
        y_conv=(tf.matmul(h_fc1, W_fc2) + b_fc2)
        
        y_conv = tf.sigmoid(y_conv)
        
#         y = tf.layers.dense(h_fc1,units=1)
        
    return y_conv
    

In [ ]:
def discriminator_task2(x,reuse=False):
    with tf.variable_scope('discriminator_task2', reuse=reuse):            
        # First conv and pool layer
        h_conv1 = conv2d_withwb(x, 3, 3, 1, 8, stride=2, name='d_conv1')
        h_conv1 = tf.contrib.layers.batch_norm(inputs=h_conv1, center=True, scale=True, is_training=True, scope='d_bn1')
        h_conv1 = tf.nn.leaky_relu(h_conv1)

        h_conv2 = conv2d_withwb(h_conv1, 3, 3, 8, 16, stride=2, name='d_conv2')
        h_conv2 = tf.contrib.layers.batch_norm(inputs=h_conv2, center=True, scale=True, is_training=True, scope='d_bn2')
        h_conv2 = tf.nn.leaky_relu(h_conv2)

        h_conv3 = conv2d_withwb(h_conv2, 3, 3, 16, 32, stride=2, name='d_conv3')
        h_conv3 = tf.contrib.layers.batch_norm(inputs=h_conv3, center=True, scale=True, is_training=True, scope='d_bn3')
        h_conv3 = tf.nn.leaky_relu(h_conv3)

        h_pool3_flat = tf.reshape(h_conv3, [-1, 4*4*32])
        h_fc1 = tf.layers.dense(h_pool3_flat, units=32, activation=tf.nn.leaky_relu)

        y = tf.layers.dense(h_fc1,units=1)
        
    return tf.nn.sigmoid(y)

In [ ]:
def discriminator_fc(x,batch_size,reuse=False):
    with tf.variable_scope('discriminator_simple', reuse=reuse):
        x = tf.reshape(x, [batch_size, -1])
        hidden_layer = tf.layers.dense(x, units=256, activation=tf.nn.leaky_relu)
        out_layer = tf.layers.dense(hidden_layer, units=1)
        out_layer = tf.nn.sigmoid(out_layer)

    return out_layer

TO DO TASK: 

Build Generator network

Input: noise vector z 

Output: sample image with dimension `28*28`

**Task 1.2**
Architecture: use `tf.nn.conv2d_transpose` to build 3 or 4 deconvolutional layer. Add `tf.nn.relu` after deconvolutional layer. Use `tf.nn.tahn` as activation function for the last layer.

**Task 2.2**
Add batch normalization after each deconvolutional layer using function `tf.contrib.layers.batch_norm`. And add `tf.nn.relu` after batch normalizatoin.  

**Task 3.2**
Build network with ONLY 2 or 3 fully connected layer (`tf.layers.dense`). Reshape the output of last layer to `[batch_size,28,28,1]`

In [ ]:
def generator_deconv(z, batch_size, z_dim,reuse=False):
    with tf.variable_scope('generator_new', reuse=reuse):
        g_dim = 64 # numbers of filters of first layer of generator
        c_dim = 1 # color dimension of output (MNIST is grayscale, so c_dim = 1 for us)
        s = 28 # output size of the image
        
        h0 = tf.reshape(z, [batch_size, 2, 2, 25]) 
        h0 = tf.nn.relu(h0)
        
        output_shape = [batch_size, 4, 4, g_dim*8]
        h_deconv1 = deconv2d(h0,output_shape=output_shape,name='g_deconv1')
        h_deconv1 = tf.contrib.layers.batch_norm(inputs=h_deconv1, center=True,scale=True,is_training=True,scope='g_bn1')
        h_deconv1 = tf.nn.relu(h_deconv1)
        
        output_shape = [batch_size, 7, 7, g_dim*4]
        h_deconv2 = deconv2d(h_deconv1,output_shape=output_shape,name='g_deconv2')
        h_deconv2 = tf.contrib.layers.batch_norm(inputs=h_deconv2, center=True, scale=True, is_training=True, scope="g_bn2")
        h_deconv2 = tf.nn.relu(h_deconv2)
        
        output_shape = [batch_size, 14, 14, g_dim*2]
        h_deconv3 = deconv2d(h_deconv2,output_shape=output_shape,name='g_deconv3')
        h_deconv3 = tf.contrib.layers.batch_norm(inputs=h_deconv3, center=True, scale=True, is_training=True, scope="g_bn3")
        h_deconv3 = tf.nn.relu(h_deconv3)
        
        output_shape = [batch_size, 28, 28, g_dim*1]
        h_deconv4 = deconv2d(h_deconv3,output_shape=output_shape,name='g_deconv4')
        h_deconv4 = tf.contrib.layers.batch_norm(inputs=h_deconv4, center=True, scale=True, is_training=True, scope="g_bn4")
        h_deconv4 = tf.nn.relu(h_deconv4)
        
        output_shape = [batch_size, 28, 28, 1]
        h_deconv4 = deconv2d(h_deconv4,s=1,output_shape=output_shape,name='g_deconv5')
        h_deconv4 = tf.contrib.layers.batch_norm(inputs=h_deconv4, center=True, scale=True, is_training=True, scope="g_bn5")
        
    return tf.nn.tanh(h_deconv4)

In [ ]:
def generator_fc(x,batch_size, z_dim,reuse=False):
    with tf.variable_scope('generator_simple',reuse=reuse):
        hidden_layer = tf.layers.dense(x, units=256, activation=tf.nn.relu)
        out_layer = tf.layers.dense(hidden_layer,units=28*28)
        out_layer = tf.nn.sigmoid(out_layer)
        out_layer = tf.reshape(out_layer,[batch_size,28,28,1])
    return out_layer

In [ ]:
def generator(z, batch_size, z_dim,reuse=False):
    with tf.variable_scope('generator', reuse=reuse):        
        g_dim = 64 # numbers of filters of first layer of generator
        c_dim = 1 # color dimension of output (MNIST is grayscale, so c_dim = 1 for us)
        s = 28 # output size of the image
        # We want to slowly upscale the image, so these values will help make that change gradual.
        s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)

        h0 = tf.reshape(z, [batch_size, 2, 2, 25]) # tf.reshape(z, [batch_size, s16+1, s16+1, -1])
        h0 = tf.nn.relu(h0)
        # Dimension of h0: batch_size x 2 x 2 x 25. This is because the input dimension is 100.
        
        # First conv layer
        output1_shape = [batch_size,4,4,g_dim*4]
        W_conv1 = tf.get_variable('g_wconv1', [4, 4, output1_shape[-1], int(h0.get_shape()[-1])], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv1 = tf.get_variable('g_bconv1', [output1_shape[-1]], 
                                  initializer=tf.constant_initializer(0.1))
        H_conv1 = tf.nn.conv2d_transpose(h0, W_conv1, output_shape=output1_shape, 
                                         strides=[1,2,2,1], padding='SAME')\
                + b_conv1
        H_conv1 = tf.contrib.layers.batch_norm(inputs=H_conv1, center=True, scale=True, is_training=True, scope='g_bn1')
        H_conv1 = tf.nn.relu(H_conv1)
        #Dimensions of H_conv1 = batch_size x 3 x 3 x 256
        
        
        #Second DeConv Layer
        output2_shape = [batch_size, 7 , 7 , g_dim*2]
        W_conv2 = tf.get_variable('g_wconv2', [4, 4, output2_shape[-1], int(H_conv1.get_shape()[-1])], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv2 = tf.get_variable('g_bconv2', [output2_shape[-1]], 
                                  initializer=tf.constant_initializer(.1))
        H_conv2 = tf.nn.conv2d_transpose(H_conv1, W_conv2, output_shape=output2_shape, 
                                         strides=[1, 2, 2, 1], padding='SAME') \
                + b_conv2
        H_conv2 = tf.contrib.layers.batch_norm(inputs = H_conv2, center=True, scale=True, is_training=True, scope="g_bn2")
        H_conv2 = tf.nn.relu(H_conv2)
        #Dimensions of H_conv2 = batch_size x 6 x 6 x 128

        #Third DeConv Layer
        output3_shape = [batch_size, 14 , 14 , g_dim*1]
        W_conv3 = tf.get_variable('g_wconv3', [4, 4, output3_shape[-1], int(H_conv2.get_shape()[-1])], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv3 = tf.get_variable('g_bconv3', [output3_shape[-1]], 
                                  initializer=tf.constant_initializer(.1))
        H_conv3 = tf.nn.conv2d_transpose(H_conv2, W_conv3, output_shape=output3_shape, 
                                         strides=[1, 2, 2, 1], padding='SAME')\
                + b_conv3
        H_conv3 = tf.contrib.layers.batch_norm(inputs = H_conv3, center=True, scale=True, is_training=True, scope="g_bn3")
        H_conv3 = tf.nn.relu(H_conv3)
        #Dimensions of H_conv3 = batch_size x 12 x 12 x 64
        
        #Fourth DeConv Layer
        output4_shape = [batch_size, s, s, c_dim]
        W_conv4 = tf.get_variable('g_wconv4', [4, 4, output4_shape[-1], int(H_conv3.get_shape()[-1])], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv4 = tf.get_variable('g_bconv4', [output4_shape[-1]], 
                                  initializer=tf.constant_initializer(.1))
        H_conv4 = tf.nn.conv2d_transpose(H_conv3, W_conv4, output_shape=output4_shape, 
                                         strides=[1, 2, 2, 1], padding='SAME')\
                + b_conv4
        H_conv4 = tf.nn.tanh(H_conv4)
        #Dimensions of H_conv4 = batch_size x 28 x 28 x 1

    return H_conv4

### Setup the inputs

In [ ]:
batch_size = 16

tf.reset_default_graph() # Since we changed our batch size (from 1 to 16), we need to reset our Tensorflow graph
z_dimensions = 100

x_placeholder = tf.placeholder("float", shape = [batch_size,28,28,1]) #Placeholder for input images to the discriminator
z_placeholder = tf.placeholder(tf.float32, [batch_size, z_dimensions]) #Placeholder for input noise vectors to the generator

Call generator to get sample data. 

In [ ]:

Dx = discriminator_fc(x_placeholder,batch_size,reuse=False) # Dx will hold discriminator prediction probabilities for the real MNIST images
Gz = generator_fc(z_placeholder, batch_size, z_dimensions) # Gz holds the generated images
Dg = discriminator_fc(Gz,batch_size, reuse=True) # Dg will hold discriminator prediction probabilities for generated images

TO DO TASK: complete the generator loss and discriminator loss according to the objective function.

Generator loss: ![generator loss](../images/generator_loss.png)

Discriminator loss: ![discriminator loss](../images/discriminator_loss.png)

Note that: when discriminator is updated, the generator is fixed; And when generator is updated, the discriminator is fixed.


In [ ]:
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.ones_like(Dg))) # ensure forward compatibility: function needs to have logits and labels args explicitly used

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dx, labels = tf.ones_like(Dx)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.zeros_like(Dg)))
d_loss = d_loss_real + d_loss_fake

In [ ]:
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'discriminator' in var.name]
g_vars = [var for var in tvars if 'generator' in var.name]
print(d_vars)

In [ ]:
adam = tf.train.AdamOptimizer()
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(d_loss, var_list=d_vars)
trainerG =  tf.train.AdamOptimizer(learning_rate=0.0002).minimize(g_loss, var_list=g_vars)

In [ ]:
iterations = 50000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(iterations):
        z_batch = np.random.uniform(-1, 1, size=[batch_size, z_dimensions])
        real_image_batch = mnist.train.next_batch(batch_size)
        real_image_batch = np.reshape(real_image_batch[0], [batch_size,28,28,1])
        _,d_loss_value = sess.run([trainerD, d_loss], # Update the discriminator
                           feed_dict={z_placeholder: z_batch,
                                      x_placeholder: real_image_batch}) 
        _,g_loss_value, g = sess.run([trainerG, g_loss, Gz],  # Update the generator
                             feed_dict={z_placeholder:z_batch})

        if i%2000==1:
            print('g_loss: ',g_loss_value, 'd_loss: ',d_loss_value)
            n = int(np.sqrt(batch_size))
            canvas_orig = np.empty((28 * n, 28 * n))
            canvas_recon = np.empty((28 * n, 28 * n))
            for i in range(n):
                # Display original images
                for j in range(n):
                    # Draw the original digits
                    canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = \
                        real_image_batch[n*i+j].reshape([28, 28])
                # Display reconstructed images
                for j in range(n):
                    # Draw the reconstructed digits
                    canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = \
                        g[n*i+j].reshape([28, 28])

            fig, axes = plt.subplots(1, 2, figsize=(2*n,n), frameon=False)
            axes[0].imshow(canvas_orig, origin="upper", cmap="gray")
            axes[0].set_title("Original Images")

            axes[1].imshow(canvas_recon, origin="upper", cmap="gray")
            axes[1].set_title("Reconstructed Images")

            axes[0].set_axis_off(); axes[1].set_axis_off()
            plt.show(fig)

In [ ]:
# sample_image = generator(z_placeholder, 1, z_dimensions, reuse=True)
# z_batch = np.random.uniform(-1, 1, size=[1, z_dimensions])
# temp = (sess.run(sample_image, feed_dict={z_placeholder: z_batch}))
# my_i = temp.squeeze()
# # plt.imshow(my_i, cmap='gray_r')
